In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np

In [26]:
players = 'data/players.csv'
players_df = pd.read_csv(players)
players_df.fillna(0)

,nflId,height,weight,birthDate,collegeName,Position,displayName
0,42901,6-1,208,1992-07-25,James Madison,SS,Dean Marlowe
1,43501,6-0,220,1994-08-11,Central Michigan,FS,Kavon Frazier
2,43534,6-0,205,1993-06-02,Illinois,SS,Clayton Fejedelem
3,43535,6-1,235,1992-12-22,Temple,MLB,Tyler Matakevich
4,44174,6-1,236,1993-02-22,Texas Tech,ILB,Sam Eguavoen
...,...,...,...,...,...,...,...
2727,33216,6-3,320,06/16/1986,Central Florida,G,Josh Sitton
2728,33184,6-3,265,05/02/1985,Winston-Salem State,DE,William Hayes
2729,45651,5-9,198,06/16/1993,Iowa,WR,Riley McCarron
2730,41284,73,230,1992-10-20,LSU,RB,Jeremy Hill


In [12]:
players_df['birthDate'] = pd.to_datetime(players_df['birthDate'], infer_datetime_format = True) 
players_df['birthDate'] = np.round((pd.Timestamp.now() - players_df['birthDate']).dt.days/365)
players_df = players_df.rename(columns={'birthDate': 'age'})

position= "SS"
plot_title_height = "Height of "+ position
plot_title_weight = "Weight of "+ position
filtered_df = players_df.loc[players_df['Position']== position]
players_heights = filtered_df["height"] # Get the Height data from DataFrame
players_heights = players_heights.apply(lambda x: x.split("-")) # Split the heights by hyphen ("-")

# Convert Heights to Centimeters and add them to DataFrame
filtered_df["height"] = players_heights.apply(lambda x: int(x[0]) * 12 + int(x[1]) if len(x) == 2 else int(x[0])) * 2.54

# Convert Weights to Kilograms and them to DataFrame
filtered_df["weight"] = round(players_df.weight * 0.453592, 2)
print(filtered_df)
fig = px.bar(filtered_df, x = filtered_df['displayName'], y = filtered_df['height'], title = plot_title_height)
fig.show()
fig = px.bar(filtered_df, x = filtered_df['displayName'], y = filtered_df['weight'], title = plot_title_weight)
fig.show()

      nflId  height  weight   age        collegeName Position  \
0     42901  185.42   94.35  29.0      James Madison       SS   
2     43534  182.88   92.99  29.0           Illinois       SS   
45    40166  182.88   86.64  31.0       Oregon State       SS   
77    46378  182.88   91.63  26.0  Mississippi State       SS   
115   52939  190.50   95.25  26.0             Temple       SS   
...     ...     ...     ...   ...                ...      ...   
2565  44862  182.88   90.26  26.0          Texas A&M       SS   
2589  35929  187.96   98.88  34.0             Toledo       SS   
2684  41381  180.34   88.45  30.0    San Diego State       SS   
2707  37175  180.34   92.99  33.0     North Carolina       SS   
2712  37168  187.96   92.08  33.0         California       SS   

            displayName  
0          Dean Marlowe  
2     Clayton Fejedelem  
45         Jordan Poyer  
77            J.T. Gray  
115        Sam Franklin  
...                 ...  
2565       Justin Evans  
2589       

/var/folders/dw/_fr9bn4s55b_z7cjb3sd9hw00000gn/T/ipykernel_2032/573946071.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/dw/_fr9bn4s55b_z7cjb3sd9hw00000gn/T/ipykernel_2032/573946071.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [17]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
plays ='data/plays.csv'
plays_df = pd.read_csv(plays)
plays_df.fillna(0)

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,specialTeamsPlayType,specialTeamsResult,kickerId,...,penaltyCodes,penaltyJerseyNumbers,penaltyYards,preSnapHomeScore,preSnapVisitorScore,passResult,kickLength,kickReturnYardage,playResult,absoluteYardlineNumber
0,2018090600,37,J.Elliott kicks 65 yards from PHI 35 to end zo...,1,0,0,PHI,Kickoff,Touchback,44966.0,...,0,0,0.0,0,0,0,66.0,0.0,40,45
1,2018090600,366,"(9:20) C.Johnston punts 56 yards to ATL 36, Ce...",1,4,4,PHI,Punt,Return,45603.0,...,UNSd,PHI 18,-15.0,0,0,0,56.0,5.0,36,18
2,2018090600,658,"(5:03) M.Bryant 21 yard field goal is GOOD, Ce...",1,4,3,ATL,Field Goal,Kick Attempt Good,27091.0,...,0,0,0.0,0,0,0,21.0,0.0,0,13
3,2018090600,677,M.Bosher kicks 64 yards from ATL 35 to PHI 1. ...,1,0,0,ATL,Kickoff,Return,37267.0,...,0,0,0.0,0,3,0,64.0,30.0,34,75
4,2018090600,872,"(:33) C.Johnston punts 65 yards to end zone, C...",1,4,18,PHI,Punt,Touchback,45603.0,...,0,0,0.0,0,3,0,65.0,0.0,45,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19974,2021010315,3683,J.Myers kicks 65 yards from SEA 35 to end zone...,4,0,0,SEA,Kickoff,Touchback,41175.0,...,0,0,0.0,16,19,0,75.0,0.0,40,75
19975,2021010315,3870,"J.Myers extra point is GOOD, Center-T.Ott, Hol...",4,0,0,SEA,Extra Point,Kick Attempt Good,41175.0,...,0,0,0.0,16,25,0,0.0,0.0,0,25
19976,2021010315,3886,J.Myers kicks 65 yards from SEA 35 to end zone...,4,0,0,SEA,Kickoff,Touchback,41175.0,...,0,0,0.0,16,26,0,75.0,0.0,40,75
19977,2021010315,4166,"T.Vizcaino extra point is GOOD, Center-C.Holba...",4,0,0,SF,Extra Point,Kick Attempt Good,47590.0,...,0,0,0.0,22,26,0,0.0,0.0,0,95


In [ ]:
x = np.array(plays_df[['playId','quarter', 'down', 'yardsToGo', 'kickerId', 'penaltyYards', 'preSnapHomeScore', 'kickLength', 'absoluteYardlineNumber']].values.tolist())
x = np.nan_to_num(x)
y = np.array(plays_df['playResult'].values.tolist())
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.66)

In [20]:
model = LinearRegression().fit(x,y)
predicted_y = model.predict(x_test)
print('intercept: ',model.intercept_, "\n")
print('slope: \n',model.coef_, "\n")
print(np.round(predicted_y))

intercept:  -5.01469688593933 

slope: 
 [-2.43807983e-03  1.82287538e+00  6.76387035e-01  1.16248815e-01
  9.32323477e-05  1.31911354e+00  6.02097301e-02  5.91721982e-01
 -2.49419762e-03] 

[39. 39. 41. ... 26. 25.  3.]


In [29]:
tracking_2020= 'data/tracking2020.csv'
tracking_df = pd.read_csv(tracking_2020)
tracking_df.fillna(0)

,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,team,frameId,gameId,playId,playDirection
0,2021-01-03T18:03:02.200,61.21,46.77,0.08,0.13,0.01,186.21,144.73,None,42901.0,Dean Marlowe,31.0,SS,home,1,2021010300,40,left
1,2021-01-03T18:03:02.300,61.22,46.77,0.11,0.19,0.01,184.87,126.09,None,42901.0,Dean Marlowe,31.0,SS,home,2,2021010300,40,left
2,2021-01-03T18:03:02.400,61.23,46.76,0.10,0.18,0.01,183.38,108.95,None,42901.0,Dean Marlowe,31.0,SS,home,3,2021010300,40,left
3,2021-01-03T18:03:02.500,61.24,46.76,0.11,0.22,0.01,180.24,91.57,None,42901.0,Dean Marlowe,31.0,SS,home,4,2021010300,40,left
4,2021-01-03T18:03:02.600,61.25,46.76,0.05,0.20,0.01,172.44,119.91,None,42901.0,Dean Marlowe,31.0,SS,home,5,2021010300,40,left
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11821696,2020-09-15T05:22:39.200,117.23,32.02,1.13,0.85,0.12,0.00,0.00,None,0.0,football,0.0,0,football,81,2020091401,4280,right
11821697,2020-09-15T05:22:39.300,117.31,32.10,1.17,0.86,0.12,0.00,0.00,None,0.0,football,0.0,0,football,82,2020091401,4280,right
11821698,2020-09-15T05:22:39.400,117.40,32.18,1.14,0.62,0.11,0.00,0.00,touchback,0.0,football,0.0,0,football,83,2020091401,4280,right
11821699,2020-09-15T05:22:39.500,117.48,32.25,1.05,0.43,0.11,0.00,0.00,None,0.0,football,0.0,0,football,84,2020091401,4280,right


In [30]:
#players_df['birthDate'] = pd.to_datetime(players_df['birthDate'], infer_datetime_format = True) 
#players_df['birthDate'] = np.round((pd.Timestamp.now() - players_df['birthDate']).dt.days/365)
#players_df = players_df.rename(columns={'birthDate': 'age'})


###Difference between SS, change if you want other positions#####
query_string = 'Position == "SS"'
plot_title = "Number of unique plays by SS in 2020 season"

pos= players_df.query(query_string)

filtered_df = tracking_df.loc[tracking_df['nflId'].isin(pos['nflId'])]
filtered_df = filtered_df[["nflId","playId"]]
df_temp = filtered_df.groupby(['nflId','playId'])
df_unique = df_temp['nflId'].count()
print(df_unique)
filtered_df = filtered_df.rename({'playId' : 'nUniquePlays'}, axis = 1)
df_unique = filtered_df.groupby('nflId').nunique()

print(df_unique)
y_axis = df_unique.reset_index()['nUniquePlays'].tolist()
x_axis = df_unique.reset_index()['nflId'].tolist()

fig = px.bar(x = x_axis,y = y_axis,
    labels ={
        "x": "NFL ID",
        "y": "Number of unique plays"
    }, title = plot_title)
fig.update_xaxes(type='category')
fig.show()


nflId    playId
34465.0  174        96
         222        48
         379        53
         387        42
         401        61
                  ... 
53081.0  2624       57
         2894      103
         3093      101
         3345       82
         4899       92
Name: nflId, Length: 8624, dtype: int64
         nUniquePlays
nflId                
34465.0            66
35583.0           183
36060.0           252
38559.0            70
38578.0           179
...               ...
52594.0           166
52939.0           146
53013.0             7
53022.0            89
53081.0            14

[77 rows x 1 columns]
